# WiDS Datathon 2023

[changeFunc.py](https://github.com/eugpoon/projects/tree/master/changeFunc.py)

#### Dependent Variable
<details>
    <summary> (click to expand)</summary>

- **contest-tmp2m-14d__tmp2m**: the arithmetic mean of the max and min observed temperature over the next 14 days for each location and start date, computed as (measured max temperature + measured mini temperature) / 2

</details>


#### Independent Variables
<details>
    <summary> (click to expand)</summary>

- **contest-slp-14d**: file containing sea level pressure (slp)
- **nmme0-tmp2m-34w**: file containing most recent monthly NMME model forecasts for tmp2m (**cancm30, cancm40, ccsm30, ccsm40, cfsv20, gfdlflora0, gfdlflorb0, gfdl0, nasa0, nmme0mean**) and average forecast across those models (nmme0mean)
- **contest-pres-sfc-gauss-14d**: pressure
- **mjo1d**: MJO phase and amplitude
- **contest-pevpr-sfc-gauss-14d**: potential evaporation
- **contest-wind-h850-14d**: geopotential height at 850 millibars
- **contest-wind-h500-14d**: geopotential height at 500 millibars
- **contest-wind-h100-14d**: geopotential height at 100 millibars
- **contest-wind-h10-14d**: geopotential height at 10 millibars
- **contest-wind-vwnd-925-14d**: longitudinal wind at 925 millibars
- **contest-wind-vwnd-250-14d**: longitudinal wind at 250 millibars
- **contest-wind-uwnd-250-14d**: zonal wind at 250 millibars
- **contest-wind-uwnd-925-14d**: zonal wind at 925 millibars
- **contest-rhum-sig995-14d**: relative humidity
- **contest-prwtr-eatm-14d**: precipitable water for entire atmosphere
- **nmme-prate-34w**: weeks 3-4 weighted average of monthly NMME model forecasts for precipitation
- **nmme-prate-56w**: weeks 5-6 weighted average of monthly NMME model forecasts for precipitation
- **nmme0-prate-56w**: weeks 5-6 weighted average of most recent monthly NMME model forecasts for precipitation
- **nmme0-prate-34w**: weeks 3-4 weighted average of most recent monthly NMME model forecasts for precipitation
- **nmme-tmp2m-34w**: weeks 3-4 weighted average of most recent monthly NMME model forecasts for target label, contest-tmp2m-14d__tmp2m
- **nmme-tmp2m-56w**: weeks 5-6 weighted average of monthly NMME model forecasts for target label, contest-tmp2m-14d__tmp2m
- **mei**: MEI (mei), MEI rank (rank), and Niño Index Phase (nip)
- **elevation**: elevation
- **contest-precip-14d**: measured precipitation
- **climateregions**: Köppen-Geigerclimateclassifications, string


- **lat**: latitude of location (anonymized)
- **lon**: longitude of location (anonymized)
- **startdate**: startdate of the 14 day period
- **sst**: sea surface temperature
- **icec**: sea ice concentration
- **cancm30, cancm40, ccsm30, ccsm40, cfsv20, gfdlflora0, gfdlflorb0, gfdl0, nasa0, nmme0mean**: most recent forecasts from weather models

</details>

In [ ]:
%matplotlib inline
import os
import random
import warnings
from zipfile import ZipFile

import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import HTML, Javascript, display
from catboost import CatBoostRegressor
from fastai.tabular.core import df_shrink
from lightgbm import LGBMRegressor
from matplotlib import pyplot as plt
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBRegressor

from changeFunc import (
    downloadData,
    corr,
    drop_outliers,
    feature_engineering,
    transform,
)


plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['figure.dpi'] = 400
plt.rcParams['font.size'] = 5
plt.rcParams['figure.titlesize'] = 10
plt.rcParams['axes.linewidth'] = 0.1
plt.rcParams['patch.linewidth'] = 0

display(HTML("<style>.container { width:100% !important; }</style>"))
random.seed(100)
warnings.filterwarnings('ignore')

### Load and Clean Data

In [ ]:
if os.path.isfile('data/widsdatathon2023pkl.zip') == False:
    !kaggle competitions download -c widsdatathon2023
    downloadData()
    !rm widsdatathon2023.zip *.pkl

z = ZipFile('data/widsdatathon2023pkl.zip')
train = df_shrink(pd.read_pickle(z.open('train_data.pkl')))
test = df_shrink(pd.read_pickle(z.open('test_data.pkl')))
submit = df_shrink(pd.read_pickle(z.open('sample_solution.pkl')))
target = 'contest-tmp2m-14d__tmp2m'
print(f'{train.isna().any().sum()} cols with null: \
        {train.columns[train.isna().any()].tolist()}')
print(f'cat vars: {list(train.select_dtypes(exclude=np.number).columns)}')

### Feature Engineering and Exploratory Data Analysis

In [ ]:
X, y, X_test, y_test = feature_engineering(train.copy(), test.copy())

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                                  random_state=100)
print(f'train: {X_train.shape} | val: {X_val.shape} | test: {X_test.shape}')

In [ ]:
# w34 = list(X.columns[X.columns.str.contains('34w')])
# w56 = list(X.columns[X.columns.str.contains('56w')])
d14 = list(X.columns[X.columns.str.contains('14d')])
# non = list(X.columns[~X.columns.str.contains('_')])
# oth = list(X.columns.difference(w34+w56+d14+non))
tmp2m = list(X.columns[X.columns.str.contains('tmp2m')])
useful = tmp2m + d14

- All subsets of variables plotted as time series
- Only nmme-tmp2m-34w__ccsm3 and nmme-tmp2m-56w__ccsm3 contained dramatic dips

In [ ]:
aa = X.groupby('startdate')[tmp2m].mean()
bb = aa.diff()  # .reset_index()
cc = np.sqrt(bb.min()**2 + bb.max()**2).sort_values().tail()
err = list(np.sort(cc.index[-2:]))
display(cc)

fig, axs = plt.subplots(ncols=2, figsize=(15, 7))

sns.lineplot(data=pd.melt(aa, ignore_index=False).reset_index(), alpha=0.5,
             x='startdate', y='value', hue='variable', ax=axs[0], legend=False)

sns.lineplot(data=pd.melt(aa[err], ignore_index=False).reset_index(),
             x='startdate', y='value', hue='variable', ax=axs[1])
plt.show()

- Assume dips resulted from errored forecasts in NNME model
- Replace any values in dips with max of 2 days before and after

In [ ]:
X, y, X_test, y_test = feature_engineering(train.copy(), test.copy())
X_err = X[err+['startdate', 'coor']].copy()
off = 2

for c in X.coor.unique():
    #     print(c, end=' ')
    df = X[X.coor == c]
    for e in [0, 1]:
        temp = df[[err[e]]].diff(1).fillna(method='bfill')
        ind = list(temp.index.difference(drop_outliers(temp, thres=4).index))
        ind1, ind2 = ind[:-1], ind[1:]
        for i in range(len(ind1)):
            if np.abs(ind1[i]-ind2[i]) > 20:
                continue
            t = X_err.loc[ind1[i]-off:ind2[i]+(off-1)]
            val = t.iloc[np.r_[0:off, -off:0]][err[e]].max()
            X_err.loc[ind1[i]:ind2[i]-1, err[e]] = val

fig, axs = plt.subplots(ncols=2, figsize=(15, 7))
for i in [0, 1]:
    sns.lineplot(X_err.groupby('startdate')[err[i]].mean(),
                 dashes=[True], alpha=0.7, label='new', ax=axs[i])
    sns.lineplot(X.groupby('startdate')[err[i]].mean(),  # marker='o',
                 alpha=0.5, label='original', ax=axs[i])
    axs[i].set(title=err[i])

X[err] = X_err[err]

### Modeling: ElasticNet
Train on each coordinate separately

In [ ]:
def modelEL(alpha, l1_ratio, X, X_test, y, change=False):
    res = y[X.index].copy()
    preds = y_test.copy()

    for i in X.coor.unique():
        xTT, xtt = X[X.coor == i][useful], X_test[X_test.coor == i][useful]
        yTT = y[xTT.index]
        # scale
        if change == True:
            xTT, xtt = transform(xTT, xtt, MinMaxScaler((1, 10)))
            cols = [c + '_change' for c in useful]
            xTT[cols] = (xTT.pct_change().replace(0, np.nan)
                         .fillna(method='bfill').fillna(method='ffill'))
            xtt[cols] = (xtt.pct_change().replace(0, np.nan)
                         .fillna(method='bfill').fillna(method='ffill'))
        else:
            xTT, xtt = transform(xTT, xtt, StandardScaler())

        # model
        el = ElasticNet(alpha=alpha, l1_ratio=l1_ratio,
                        max_iter=10000, random_state=100).fit(xTT, yTT)
        preds[xtt.index] = el.predict(xtt)
        res[xTT.index] = yTT - el.predict(xTT).flatten()  # train residuals
    return res, preds

#### Variables used: **tmp2m group, 14d group**

In [ ]:
%%time
X1 = X.groupby('coor')[useful].apply(drop_outliers).reset_index(level='coor')

# alpha = [0.01, 0.1, 1]
# ratios = np.round(np.arange(0.2, 0.9, 0.1), 1)
# comp1 = pd.DataFrame(columns=['alpha', 'ratio', 'rmse'])

# for a, r in product(alpha, ratios):
#     res, preds = modelEL(a, r, X1, X_test, y, False)
#     comp1.loc[len(comp1.index)] = [a, r, np.sqrt(np.mean(res**2))]

# el1 = comp1.sort_values('rmse').head().mean()
el1 = pd.Series([0.01, 0.6, 0.574843], index=[
                'alpha', 'ratio', 'rmse'])  # above results
res1, pred1 = modelEL(el1.alpha, el1.ratio, X1, X_test, y, False)
print(el1.alpha, el1.ratio)

#### Variables used: **tmp2m group, 14d group, percent change of tmp2m**

In [ ]:
%%time
# comp2 = pd.DataFrame(columns=['alpha', 'ratio', 'rmse'])

# for a, r in product(alpha, ratios):
#     res, preds = modelEL(a, r, X, X_test, y, True)
#     comp2.loc[len(comp2.index)] = [a, r, np.sqrt(np.mean(res**2))]

# el2 = comp2.sort_values('rmse').head().mean()
el2 = pd.Series([0.01, 0.4, 0.471854], index=[
                'alpha', 'ratio', 'rmse'])  # above results
res2, pred2 = modelEL(el2.alpha, el2.ratio, X, X_test, y, True)
print(el2.alpha, el2.ratio)

### Modeling: Stacking
Train with unused columns from ElasticNet modeling

In [ ]:
def modelStack(X, y):

    estimators = [('rid', Ridge(alpha=0.6)),
                  ('las', Lasso(alpha=0.01, random_state=100)),
                  ('ela', ElasticNet(alpha=0.01, l1_ratio=0.6, random_state=100)),
                  ('lgb', LGBMRegressor(objective='regression', metric='rmse',
                                        verbose=-1, random_state=100)),
                  ('xgb', XGBRegressor(objective='reg:squarederror', eval_metric='rmse',
                                       random_state=100)),
                  ('cat', CatBoostRegressor(
                      loss_function='RMSE', verbose=0, random_seed=100))
                  ]
    reg = StackingRegressor(estimators=estimators,
                            final_estimator=CatBoostRegressor(loss_function='RMSE',
                                                              verbose=0, random_seed=100)
                            ).fit(X, y[X.index])
    return reg

In [ ]:
%%time
drop = useful + corr(X.loc[X1.index].drop(columns=useful), 0.6)
X2 = drop_outliers(X.loc[X1.index].drop(columns=drop))
X2_test = X_test[X2.columns]
X2, X2_test = transform(X2, X2_test)

pred3 = modelStack(X2, y).predict(X2_test)

### Submission

In [ ]:
submit[target] = pred1*.7 + pred2*.25 + pred3*.05
display(submit)
# submit.set_index('index').to_csv('s1.csv')

In [ ]:
# save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))
# save notebook as html to eugpoon.github.io/projects
!jupyter nbconvert change.ipynb --to html
%mv "change.html" "../eugpoon.github.io/projects/"
# restyle imports, clear output, replace file
!cleanipynb change.ipynb
# restart kernel
display(HTML("<script>Jupyter.notebook.kernel.restart()</script>"))